In [1]:
from transformers import AutoTokenizer
import pandas as pd
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

2023-12-05 13:51:41.368310: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-05 13:51:41.368331: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/rodrigo/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model_id = 'philschmid/BERT-tweet-eval-emotion'

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [13]:
#data = pd.read_csv("./drive/MyDrive/data_processed.csv")
import os

data_train = pd.read_csv("../bert_teste/datasets/train_data.csv")
data_test = pd.read_csv("../bert_teste/datasets/test_data.csv")

In [14]:
data_train

,Unnamed: 0,Sentiment,SentimentText
0,634820,0,Crazy weekend and now alotta work to do today!
1,621028,1,Back from camping. Ireland Wait a second and s...
2,1091843,1,Over and out how exciting
3,1227632,1,Sunny Sunday.
4,355435,0,||T|| sorry. *cries in a corner*
...,...,...,...
99995,1298441,0,"Tula's sick! Nausious, Nasal congestion, heada..."
99996,946626,0,is pissed because Newcastle lost against Fulha...
99997,130658,1,||T|| have you been voting for me to WIN the c...
99998,1498924,0,My break is over back to the inferno [ I Lov...


In [15]:
sentence_train_tokenized = []

for i in range(data_train.shape[0]):

  if i%50000 == 0:
    print(i)

  sentence_train_tokenized.append(tokenizer.encode(data_train.iloc[i]["SentimentText"], padding="max_length", truncation="only_first", max_length=35))

0
50000


In [16]:
sentence_test_tokenized = []

for i in range(data_test.shape[0]):

  if i%50000 == 0:
    print(i)

  sentence_test_tokenized.append(tokenizer.encode(data_test.iloc[i]["SentimentText"], padding="max_length", truncation="only_first", max_length=35))

0


In [17]:
data_train["SentencesTokenized"] = sentence_train_tokenized
data_test["SentencesTokenized"] = sentence_test_tokenized

In [18]:
data_train

,Unnamed: 0,Sentiment,SentimentText,SentencesTokenized
0,634820,0,Crazy weekend and now alotta work to do today!,"[101, 11722, 5138, 1105, 1208, 2393, 15719, 11..."
1,621028,1,Back from camping. Ireland Wait a second and s...,"[101, 4388, 1121, 15973, 119, 2270, 7911, 170,..."
2,1091843,1,Over and out how exciting,"[101, 3278, 1105, 1149, 1293, 11215, 102, 0, 0..."
3,1227632,1,Sunny Sunday.,"[101, 17321, 3625, 119, 102, 0, 0, 0, 0, 0, 0,..."
4,355435,0,||T|| sorry. *cries in a corner*,"[101, 197, 197, 157, 197, 197, 2959, 119, 115,..."
...,...,...,...,...
99995,1298441,0,"Tula's sick! Nausious, Nasal congestion, heada...","[101, 17037, 1742, 112, 188, 4809, 106, 11896,..."
99996,946626,0,is pissed because Newcastle lost against Fulha...,"[101, 1110, 9254, 1272, 7685, 1575, 1222, 2397..."
99997,130658,1,||T|| have you been voting for me to WIN the c...,"[101, 197, 197, 157, 197, 197, 1138, 1128, 115..."
99998,1498924,0,My break is over back to the inferno [ I Lov...,"[101, 1422, 2549, 1110, 1166, 1171, 1106, 1103..."


In [22]:
X_train = pd.DataFrame(sentence_train_tokenized, columns=[f'Column{i+1}' for i in range(len(sentence_train_tokenized[0]))])
X_test = pd.DataFrame(sentence_test_tokenized, columns=[f'Column{i+1}' for i in range(len(sentence_test_tokenized[0]))])

In [23]:
X_train

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35
0,101,11722,5138,1105,1208,2393,15719,1161,1250,1106,...,0,0,0,0,0,0,0,0,0,0
1,101,4388,1121,15973,119,2270,7911,170,1248,1105,...,1103,3227,13670,12549,5753,106,106,102,0,0
2,101,3278,1105,1149,1293,11215,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,101,17321,3625,119,102,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,101,197,197,157,197,197,2959,119,115,13881,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,101,17037,1742,112,188,4809,106,11896,1361,4179,...,170,14147,119,1109,1611,2087,7535,4268,1494,102
99996,101,1110,9254,1272,7685,1575,1222,23973,1105,9632,...,15969,155,21678,17020,13821,24805,14731,1592,11607,102
99997,101,197,197,157,197,197,1138,1128,1151,6612,...,150,25290,12809,4759,8413,131,120,120,4296,102
99998,101,1422,2549,1110,1166,1171,1106,1103,1107,24215,...,0,0,0,0,0,0,0,0,0,0


In [24]:
y_train = data_train["Sentiment"]
y_test = data_test["Sentiment"]

In [26]:
y_test

0       1
1       1
2       1
3       0
4       0
       ..
4280    1
4281    0
4282    1
4283    0
4284    0
Name: Sentiment, Length: 4285, dtype: int64

In [25]:
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")

y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")